In [1]:
# pip install selenium==4.23.1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 均一網址
url = "https://www.junyiacademy.org/course-compare/math-elem/math-5/k-m5a_y112"

# Get Chrome driver
driver = webdriver.Chrome()
driver.get(url)

# Give the page some time to load elements
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'topic-card-k-m5a')]")))

# File we want to write to
filename = 'textbook_links_5a.txt'
with open(filename, 'w') as file:

    # Iterate through each element, click on it, extract data, then go back
    for i in range(10):
        # 0-10 -> first semester
        # 11-20 -> second semester

        # Elements with units
        elements = driver.find_elements(
            By.CSS_SELECTOR, 
            '.MuiButtonBase-root.MuiButton-root.MuiButton-contained.MuiButton-containedPrimary.MuiButton-sizeMedium.MuiButton-containedSizeMedium.css-ll033b'
        )

        ActionChains(driver).move_to_element(elements[i]).click(elements[i]).perform()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.MuiButtonBase-root.MuiListItem-root.MuiListItem-gutters.MuiListItem-padding.MuiListItem-button.MuiListItem-secondaryAction.css-cwpkkg')))

        # Get the unit string
        unit_string_element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="side-panel-title"].css-rwiwre')
        unit_string = unit_string_element.text

        file.write(f"\n{unit_string}\n")

        # Get links to videos
        links_to_videos = driver.find_elements(By.CSS_SELECTOR, '.MuiButtonBase-root.MuiListItem-root.MuiListItem-gutters.MuiListItem-padding.MuiListItem-button.MuiListItem-secondaryAction.css-cwpkkg')
        # some links are not actually links to videos; instead, they store practice problems

        for element in links_to_videos:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.MuiTypography-root.MuiTypography-body1.css-37h3rr')))
            child_with_text = element.find_element(By.CSS_SELECTOR, 'span.MuiTypography-root.MuiTypography-body1.css-37h3rr')
            child_text = child_with_text.text
            original_url = driver.current_url
            element.click()
            WebDriverWait(driver, 10).until(lambda driver: driver.current_url != original_url)
            current_url = driver.current_url
            file.write(f"{child_text}:\n{current_url}\n")
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.MuiButtonBase-root.MuiListItem-root.MuiListItem-gutters.MuiListItem-padding.MuiListItem-button.MuiListItem-secondaryAction.css-cwpkkg')))

        driver.back()

        # Wait for the original page to load back
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.MuiButtonBase-root.MuiButton-root.MuiButton-contained.MuiButton-containedPrimary.MuiButton-sizeMedium.MuiButton-containedSizeMedium.css-ll033b')))

# Close the driver
driver.quit()